---
### VGG16 ImageNet-Mini

In [121]:
import torch
from torch.utils.data import DataLoader 

from torch import optim
from torch.optim.lr_scheduler import StepLR

import torch.nn as nn
import torch.nn.functional as f

import torchvision
from torchvision import transforms as tf, datasets 

from torchsummary import summary

import matplotlib.pyplot as plt
import numpy as np
import json

In [122]:
# 하이퍼 파라미터 지정
ImageNet_BATCH_SIZE = 64

CONV_KERNEL_SIZE = 3
MAXP_KERNEL_SIZE = 2

c_stride = 1
c_padding = 1
mp_stride = 2

lr = 0.01
step_size=30
gamma= 0.1

In [123]:

transform = tf.Compose([
    tf.Resize((224, 224)), # 이미지 사이즈 조정 
    tf.ToTensor(), # 텐서로 변환
    tf.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [124]:
# 데이터 셋 경로 지정
Path_train_ImageNet = '../dataset/imagenet-mini/train'
Path_test_ImageNet = '../dataset/imagenet-mini/val'
Path_cls_ImageNet = '../dataset/imagenet-mini/imagenet_class_index.json'


In [125]:

train_dataset = datasets.ImageFolder(Path_train_ImageNet)
test_dataset = datasets.ImageFolder(Path_test_ImageNet)

train_dataset.transform = transform
test_dataset.transform = transform
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

img, _ = train_dataset[1]
print("img shape: ", img.shape) # size = [3, 96, 96]
print("len train", len(train_dataset)) # 학습용 데이터 5000개
print("len test", len(test_dataset))   # 테스트 데이터 8000개


img shape:  torch.Size([3, 224, 224])
len train 34745
len test 3923


In [128]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Sequential(
            #3 224 128
            nn.Conv2d(3, 64, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(64, 64, 3, padding=1),nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2),
            #64 112 64
            nn.Conv2d(64, 128, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(128, 128, 3, padding=1),nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2),
            #128 56 32
            nn.Conv2d(128, 256, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(256, 256, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(256, 256, 3, padding=1),nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2),
            #256 28 16
            nn.Conv2d(256, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2),
            #512 14 8
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.Conv2d(512, 512, 3, padding=1),nn.LeakyReLU(0.2),
            nn.MaxPool2d(2, 2)
        )
        #512 7 4

        self.avg_pool = nn.AvgPool2d(7)
        #512 1 1
        self.classifier = nn.Linear(512, 1000)
        """
        self.fc1 = nn.Linear(512*2*2,4096)
        self.fc2 = nn.Linear(4096,4096)
        self.fc3 = nn.Linear(4096,10)
        """

    def forward(self, x):
        #print(x.size())
        features = self.conv(x)
        #print(features.size())
        x = self.avg_pool(features)
        #print(avg_pool.size())
        x = x.view(features.size(0), -1)
        #print(flatten.size())
        x = self.classifier(x)
        #x = self.softmax(x)
        return x, features


In [131]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = Net()
net = net.to(device)
param = list(net.parameters())

RuntimeError: CUDA error: device-side assert triggered